# **DATA VISUALIZATION**

## Objectives

- Answer business requirement 1:
       The client is interested in conducting a study to visually differentiate a cherry leaf that is healthy and that contains powdery mildew.
- This notebook also aims to validate Hypothesis 1:
We hypothesize that infected leaves have clear white marks with which they can be distinguished from healthy leaves


## Inputs

* inputs/cherry_dataset/cherry-leaves/train
* inputs/cherry_dataset/cherry-leaves/test
* inputs/cherry_dataset/cherry-leaves/validation

## Outputs

* Image shape embeddings pickle file, in this case we already know that it will be 50 x 50.
* Mean and variability of the leaves images per both labels, and plot them.
* Plot to distinguish contrast between healthy and powdery mildew infected leaves images.
* Generate code that answers business requirement 1 and can be used to build image montage on Streamlit dashboard.

## Additional Comments

* After exploring the dataset, we notice that the images are 256 pixels × 256 pixels, that should be the image shape of the files, but the trained model will be very large, that is why we consider using an image shape of 50 x 50.

---

## **Set Data Directory**

### Import Libraries

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
sns.set_style("whitegrid")
from matplotlib.image import imread